# <div style='text-align: center'>Sales Analysis</div>

## Import libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from os import listdir

#### Merge sales of 12 months into a single CSV file

In [ ]:
dataFolderPath = './Sales Data/'
dataFiles = [ f for f in listdir(dataFolderPath) if f.endswith('.csv') ]

year_data = pd.DataFrame()

for file in dataFiles:
    month_df = pd.read_csv( dataFolderPath + file )
    year_data = pd.concat([year_data, month_df])

year_data.to_csv("full_data.csv", index=False)

#### Read the full year data

In [ ]:
full_data = pd.read_csv("full_data.csv")

full_data.head()

### Task 1: The Best Month Sales and Total Sales

#### Step 1: Clean up data

##### Drop rows with nan values

In [ ]:
full_data[full_data.isnull().any(axis=1)]

full_data = full_data.dropna(how='all')

##### Remove columns with not valid date

In [ ]:
full_data = full_data[full_data['Order Date'] != 'Order Date']

#### Step 2: Convert price and quantity columns to integer

In [ ]:
full_data['Quantity Ordered'] = pd.to_numeric(full_data['Quantity Ordered'])
full_data['Price Each'] = pd.to_numeric(full_data['Price Each'])

#### Step 3: Add total price column

In [ ]:
full_data['Total Price'] = full_data['Price Each'] * full_data['Quantity Ordered']

#### Step 4: Conver order date column to datetime object

In [ ]:
full_data['Order Date'] = pd.to_datetime(full_data['Order Date'], format='%m/%d/%y %H:%M')

#### Step 5: Group by month

In [ ]:
months_sales = full_data.groupby( full_data['Order Date'].dt.month )['Total Price'].sum().reset_index(name='Sales')        
months_sales.rename(columns={'Order Date': 'Month'}, inplace=True)

#### Step 6: Show results

In [ ]:
import datetime

months = list(range(1, 13))

for i, val in enumerate(months):
    month = datetime.date(1900, val , 1).strftime('%b')
    months[i] = month

plt.bar(months, months_sales['Sales'], width=0.7)
plt.xticks(months)
plt.title('Sales of each month')
plt.xlabel('Months')
plt.ylabel('Sales in DH')

plt.gca().get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, _: f'{int(x):,}'))
plt.show()

### Task 2: The City with the Highest Sales

#### Step 1: Get cities name from the order address

In [ ]:
cities = full_data['Purchase Address'].apply(lambda x: x.split(',')[1].lstrip())

#### Step 2: Group by city

In [ ]:
cities_sales = full_data.groupby( cities )['Total Price'].sum().reset_index(name='Sales')        
cities_sales.rename(columns={'Purchase Address': 'City'}, inplace=True)

#### Step 3: Show results

In [ ]:
plt.bar(cities_sales['City'], cities_sales['Sales'], width=0.7)
plt.xticks(cities_sales['City'], rotation='vertical')
plt.title('Sales of each city')
plt.xlabel('Citites')
plt.ylabel('Sales in DH')

plt.gca().get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, _: f'{int(x):,}'))
plt.show()

### Task 3: The Best Time to Display Advertisements for Maximum Probability of Customer Purchase

#### Step 1: Group by hour

In [ ]:
hours_orders = full_data.groupby(full_data['Order Date'].dt.hour)['Order Date'].count().reset_index(name='Count')
hours_orders.rename(columns={'Order Date': 'Hour'}, inplace=True)

#### Step 2: Show results

In [ ]:
plt.plot(hours_orders['Hour'], hours_orders['Count'])
plt.xticks(hours_orders['Hour'], size=8.5)
plt.title('Sales of each hour')
plt.xlabel('Hours')
plt.ylabel('Number of Orders')

plt.grid()
plt.show()

### Task 4: Identify the Products Most Frequently Sold Together

#### Step 1: Group by orders and merge products to products column

In [ ]:
df = full_data[full_data['Order ID'].duplicated(keep=False)]

df = df.groupby('Order ID')['Product'].agg(lambda x: ','.join(x)).reset_index(name='Products')


#### Step 2: Count most sold products together

In [ ]:
from itertools import combinations
from collections import Counter

count = Counter()

for products in df['Products']:
    products_list = products.split(',')
    count.update(Counter(combinations(products_list, 2)))

#### Step 3: Extract the products data

In [ ]:
products = []
products_count= []

top5 = count.most_common(5)
for i in range(0, 5):
    productCombo = top5[i][0][0] + ' / ' +  top5[i][0][1]
    products.append(productCombo)
    products_count.append(top5[i][1])


#### Step 4: Show results

In [ ]:
plt.bar(products, products_count, width=0.6)
plt.xticks(products, rotation='vertical')
plt.title('The most sold products ( together )')
plt.xlabel('Products')
plt.ylabel('Number of Orders')

plt.show()

### Task 5: The top products based on sales

#### Step 1: Group products by product name

In [ ]:
products_sales = full_data.groupby('Product')['Total Price'].sum().reset_index(name='Sales')

#### Step 2: Show results

In [ ]:
plt.bar(products_sales['Product'], products_sales['Sales'])
plt.xticks(products_sales['Product'], rotation='vertical')
plt.title('The top products ( sales )')
plt.xlabel('Products')
plt.ylabel('Sales')

plt.gca().get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, _: f'{int(x):,}'))
plt.show()

### Task 6: The most selling products and their relation with the price

#### Step 1: Group products by product name

In [ ]:
products_quantity = full_data.groupby('Product')['Quantity Ordered'].sum().reset_index()

#### Step 2: Add the mean price column

In [ ]:
products_quantity["Price"] = full_data.groupby('Product')['Price Each'].mean().reset_index()['Price Each']

#### Step 3: Show results

In [ ]:
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()

ax1.bar(products_quantity['Product'], products_quantity['Quantity Ordered'], color='g')
ax2.plot(products_quantity['Product'], products_quantity['Price'], 'b-')

ax1.set_xticks(products_quantity['Product'])
ax1.set_xticklabels(products_quantity['Product'], rotation='vertical', size=8 )

plt.title('The most selling products')

ax1.set_xlabel('Product Name')
ax1.set_ylabel('Quantity Ordered', color='g')
ax2.set_ylabel('Price', color='b')

plt.show()

### End.